In [1]:
import sys
import pickle
sys.path.insert(0,'./')
import pyccl as ccl
# from importlib import reload

In [5]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [6]:
s=[0,4]
4 in s

True

In [7]:
from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
LC=LocalCluster(n_workers=1,processes=False,memory_limit='50gb',threads_per_worker=1,memory_spill_fraction=.99,
               memory_monitor_interval='2000ms')
client=Client(LC)

In [8]:
client

Client Scheduler: inproc://192.168.86.48/71398/1 Dashboard: http://192.168.86.48:8787/status,Cluster Workers: 1 Cores: 1 Memory: 34.36 GB


In [9]:
from cov_3X2 import *
from lsst_utils import *
# from fisher import *

In [10]:
#only for python3
import importlib
reload=importlib.reload

# Lensing X Lensing Correlation function covariance

## Multiple redshift bins (Tomography)

In [11]:
import cov_utils
reload(cov_utils)
from cov_utils import *

import window_utils
reload(window_utils)
from window_utils import *

In [12]:
import cov_3X2
reload(cov_3X2)
from cov_3X2 import *
from wigner_transform import *

In [13]:
import hankel_transform
from astropy.io import fits
reload(hankel_transform)
from hankel_transform import *


In [14]:
cosmo_params={'h': 0.8433,
 'Omb': 0.062,
 'Omd': 0.2837-0.062,
 's8': 0.795431,
 'Om': 0.2837,
 'As': 1.685e-09,
'Ase9': 1.685,
 'mnu': 0.0,
 'Omk': 0.0,
 'tau': 0.06,
 'ns': 0.96859,
 'w': -1,
 'wa': 0,
 'Oml': 1.-0.2837}
PS=Power_Spectra(cosmo_params=cosmo_params)
k,pk,s8=PS.camb_pk(z=[0],cosmo_params=cosmo_params,return_s8=True)
s8

0.7971166458167727

In [15]:
mnu=cosmo_h.m_nu*0
cosmo=cosmo.clone(m_nu=mnu,Ob0=cosmo_params['Omb'],Om0=cosmo_params['Om'])
cosmo_h=cosmo.clone(H0=100) #,m_nu=mnu,Ob0=cosmo_params['Omb'],Om0=cosmo_params['Om'])
ccl_cosmo = ccl.Cosmology(Omega_c=cosmo_params['Omd'], Omega_b=cosmo_params['Omb'], 
                      h=cosmo_params['h'], A_s=cosmo_params['As'], n_s=cosmo_params['ns'],)#m_nu=[0,0,0.06])

In [16]:
cov_des=np.genfromtxt('../covariance_testing/data//cosmolike_cov_Y3ish/cov_y3_mcal_emu_final.txt')
covG_des=np.genfromtxt('../covariance_testing/data/cosmolike_cov_Y3ish/cov_y3_mcal_emu_final.txt') 
# covNG_shear=np.genfromtxt('/home/deep/repos/DESC_SRD/cov/Y10_shear_shear_covNG',) 

l_min0=20
l_max0=15000
nl0=20
l_bins=np.logspace(np.log10(l_min0),np.log10(l_max0),nl0+1)
lb=0.5*(l_bins[1:]+l_bins[:-1])

lb15=np.tile(lb,15)
x=lb15<3000
covG_shear=covG_des[:,2].reshape(900,900)[:400,:400] #10 X 2 X 20
cov_shear=cov_des[:,2].reshape(900,900)[:400,:400]

covG_ggl=covG_des[:,2].reshape(900,900)[400:800,400:800] #10 X 2 X 20
cov_ggl=cov_des[:,2].reshape(900,900)[400:800,400:800]

covG_gg=covG_des[:,2].reshape(900,900)[800:900,800:900] #10 X 2 X 20
cov_gg=cov_des[:,2].reshape(900,900)[800:900,800:900]


cov_des=cov_des[:,2].reshape(900,900)
covG_des=covG_des[:,2].reshape(900,900)
# covNG_shear=covNG_shear[x].T[x].T

x=lb<3000
lb=lb[x]
l_bins=l_bins[:16]

In [17]:
sigma_gamma=0.3944/sqrt(2.)  #*2**0.25

In [18]:
window_lmax=30

In [19]:
l_max=4e4
l_maxW=6e4
l_min=2
# l0=np.arange(l_min,l_maxW)   #np.arange(l_min,l_max)
l0=np.logspace(np.log10(l_min),np.log10(l_maxW),6.e3)
lw=np.arange(window_lmax)
l0=np.sort(np.append(l0,lw))
# l0=np.sort(np.append(l0,np.logspace(np.log10(l_maxW),np.log10(l_maxW*10),1.e3)))
l0=np.unique(np.int32(l0))   #np.arange(l_min,l_max)
# l0=np.unique(np.int32(np.logspace(np.log10(l_min),np.log10(l_maxW),4.e3)))   #np.arange(l_min,l_max)

In [35]:
#Setup lens redshift bins
source_f='../covariance_testing/data//cosmolike_cov_Y3ish/lens.nz'
zmin=0
zmax=3.5
f_sky=5000./(4*np.pi/d2r**2)

nl_bins=4  # Number of tomographic bins
zl_bins=DES_lens_bins(fname=source_f,l=[1],f_sky=f_sky)

{('galaxy', 'galaxy'): [(0, 0), (1, 1), (2, 2), (3, 3)]} {'shear': 0, 'galaxy': 5, 'kappa': 0}
Win gen: step size 768
Window done
cl dict done


In [21]:
def DES_bins(fname='~/Cloud/Dropbox/DES/2pt_NG_mcal_final_7_11.fits',l=None,sigma_gamma=0,nside=256,mask_start_pix=0,window_cl_fact=0,unit_win=True,use_window=True,f_sky=1):
    z_bins={}
#     try:
#         t=Table.read(fname,format='fits',hdu=6)
#         dz=t['Z_HIGH']-t['Z_LOW']
#         zmax=max(t['Z_HIGH'])
#     except:
    t=np.genfromtxt(fname,names=('Z_MID','BIN1','BIN2','BIN3','BIN4'))
    dz=np.gradient(t['Z_MID'])
    zmax=max(t['Z_MID'])+dz[-1]/2.
    print(t)
    nz_bins=4
    nz=[1.496,1.5189,1.5949,0.7949]

    z_bins['SN']={}
    z_bins['SN']['galaxy']=np.zeros((len(l),nz_bins,nz_bins))
    z_bins['SN']['shear']=np.zeros((len(l),nz_bins,nz_bins))

    for i in np.arange(nz_bins):
        z_bins[i]={}
        z_bins[i]['z']=t['Z_MID']
        z_bins[i]['dz']=dz
        z_bins[i]['nz']=nz[i]
        z_bins[i]['pz']=t['BIN'+str(i+1)]
        z_bins[i]['W']=1.
        z_bins[i]['AI']=0
        z_bins[i]['AI_z']=0
        z_bins[i]['pzdz']=z_bins[i]['pz']*z_bins[i]['dz']
        z_bins[i]['Norm']=np.sum(z_bins[i]['pzdz'])
        #z_bins['SN']['galaxy'][:,i,i]=galaxy_shot_noise_calc(zg1=z_bins[i],zg2=z_bins[i])
        z_bins['SN']['shear'][:,i,i]=shear_shape_noise_calc(zs1=z_bins[i],zs2=z_bins[i],
                                                            sigma_gamma=sigma_gamma)
        z_bins[i]['lm']=1.e7
        z_bins[i]['b1']=1
        z_bins[i]['AI']=0
        z_bins[i]['AI_z']=0
        z_bins[i]['mag_fact']=0
    z_bins['n_bins']=nz_bins
    z_bins['nz']=nz
    z_bins['zmax']=zmax
    if use_window:
        z_bins=set_window(zs_bins=z_bins,f_sky=f_sky,nside=nside,mask_start_pix=mask_start_pix,
                           window_cl_fact=window_cl_fact,unit_win=unit_win)
    return z_bins

In [22]:
#Setup source redshift bins
source_f='../covariance_testing/data/cosmolike_cov_Y3ish/source.nz'
zmin=0
zmax=3.5


nbins=4  # Number of tomographic bins
zs_bins=DES_bins(fname=source_f,sigma_gamma=sigma_gamma,l=[1],f_sky=f_sky)

[(1.0000e-04, 1.00785069e-02, 3.30870357e-03, 3.73129507e-03, 6.77184900e-03)
 (1.0100e-02, 3.06853286e-02, 1.13658813e-02, 8.79425375e-03, 1.88871101e-02)
 (2.0100e-02, 5.26837433e-02, 2.01429443e-02, 1.33825601e-02, 2.87010006e-02)
 (3.0100e-02, 8.13730497e-02, 3.12457906e-02, 1.85378123e-02, 4.05252839e-02)
 (4.0100e-02, 1.12901768e-01, 4.14764681e-02, 2.49324346e-02, 5.30902381e-02)
 (5.0100e-02, 1.50249875e-01, 5.82830210e-02, 3.29487858e-02, 6.98611454e-02)
 (6.0100e-02, 1.89256649e-01, 7.49096025e-02, 4.25868660e-02, 8.57591190e-02)
 (7.0100e-02, 2.27757389e-01, 9.85135338e-02, 5.46509472e-02, 1.06603717e-01)
 (8.0100e-02, 2.69266219e-01, 1.24318653e-01, 6.39330382e-02, 1.27421862e-01)
 (9.0100e-02, 3.17648674e-01, 1.53446319e-01, 8.00316646e-02, 1.55831885e-01)
 (1.0010e-01, 3.84192122e-01, 1.90243533e-01, 9.57347474e-02, 1.84268360e-01)
 (1.1010e-01, 4.67504968e-01, 2.32398355e-01, 1.12716755e-01, 2.04160666e-01)
 (1.2010e-01, 5.64044976e-01, 2.68544903e-01, 1.21629671e-01, 2.

./cov_utils.py:60: RuntimeWarning: invalid value encountered in true_divide
  Win0=2*jn(1,l_th)/l_th
./angular_power_spectra.py:23: RuntimeWarning: divide by zero encountered in true_divide
  self.cl_f=(l+0.5)**2/(l*(l+1.)) # cl correction from Kilbinger+ 2017


cl dict done


In [23]:
# f_sky=4182./(4*np.pi/d2r**2)

In [24]:
f_sky*4*np.pi

1.5230870989335428

In [25]:
do_xi=True
bin_xi=True
do_cov=True
bin_cl=False
th_min=2.5/60
th_max=250./60
n_th_bins=20
th_bins=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins+1)
th=np.logspace(np.log10(th_min*0.98),np.log10(1),n_th_bins*30)
th2=np.linspace(1,th_max*1.02,n_th_bins*30)
# th2=np.logspace(np.log10(1),np.log10(th_max),60*6)
th=np.unique(np.sort(np.append(th,th2)))
thb=np.sqrt(th_bins[1:]*th_bins[:-1])

In [26]:
#Hankel Transform setup
prune_theta=0
HT_kwargs={'l_min':l_min,  'l_max':l_max,
                        'theta_min':th_min*d2r*.9, 'theta_max':th_max*d2r,
                        'n_zeros':40000, 'prune_theta':prune_theta, 'm1_m2':[(2,2),(2,-2),(0,2),(0,0)]}
# %time HT=hankel_transform(**HT_kwargs)
# HT_kwargs2={'l_min':l_min,  'l_max':l_max,
#                         'theta_min':th_min*d2r*.9, 'theta_max':th_max*d2r,
#                         'n_zeros':40000, 'prune_theta':2, 'm1_m2':[(2,-2)]}#,(2,-2)]}

In [27]:
use_window=False
do_sample_variance=True
SSV_cov=False
tidal_SSV=True


In [28]:
import cov_utils
reload(cov_utils)
from cov_utils import *
import cov_3X2
reload(cov_3X2)
from cov_3X2 import *


In [29]:
power_spectra_kwargs={'cosmo':cosmo,'cosmo_params':cosmo_params,'pk_func':'camb_pk_too_many_z'}

In [61]:
corr_ggl=('shear','galaxy')
corr_gg=('galaxy','galaxy')
corr_ll=('shear','shear')
m1m2=(0,2)
m1m2_gg=(0,0)

corrs=[corr_ll,corr_ggl,corr_gg]

In [62]:
store_win=True
xi_win=True
# if xi_win:
#     corrs=[corr_ll] #memory issues

In [36]:
#### Wigner Transform setup... Using high ell to test convergence
WT_L_kwargs={'l': l0,'theta': th*d2r,'m1_m2':[(2,2),(2,-2),(0,2),(2,0),(0,0)]}
%time WT_L=wigner_transform(**WT_L_kwargs)

./wigner_functions.py:39: RuntimeWarning: invalid value encountered in true_divide
  d_mat/=np.sqrt(binom(k+b,b))
./wigner_functions.py:39: RuntimeWarning: divide by zero encountered in true_divide
  d_mat/=np.sqrt(binom(k+b,b))
./wigner_functions.py:39: RuntimeWarning: divide by zero encountered in true_divide
  d_mat/=np.sqrt(binom(k+b,b))


CPU times: user 2.03 s, sys: 1.14 s, total: 3.17 s
Wall time: 36.8 s


In [37]:
do_cov=False

In [63]:
XI_W_L= cov_3X2(zs_bins=zs_bins,l=l0,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,zg_bins=zl_bins,
                do_sample_variance=do_sample_variance,use_window=use_window,
#                 power_spectra_kwargs=power_spectra_kwargs,
                corrs=corrs,
                store_win=store_win,window_lmax=window_lmax,
               SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV,do_xi=True,HT=WT_L,f_sky=f_sky,
               bin_xi=bin_xi,theta_bins=th_bins,
               )#xi_win=xi_win)#ns=np.inf)

{('shear', 'shear'): [(0, 0), (0, 1), (0, 2), (0, 3), (1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)], ('galaxy', 'galaxy'): [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)], ('shear', 'galaxy'): [(0, 0), (1, 0), (2, 0), (3, 0), (0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (3, 2), (0, 3), (1, 3), (2, 3), (3, 3), (0, 4), (1, 4), (2, 4), (3, 4)], ('galaxy', 'shear'): [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3)]} {'shear': 4, 'galaxy': 5, 'kappa': 0}
Win gen: step size 1874
Window done


./cov_utils.py:60: RuntimeWarning: invalid value encountered in true_divide
  Win0=2*jn(1,l_th)/l_th
./angular_power_spectra.py:23: RuntimeWarning: divide by zero encountered in true_divide
  self.cl_f=(l+0.5)**2/(l*(l+1.)) # cl correction from Kilbinger+ 2017


In [64]:
corrs

[('shear', 'shear'), ('shear', 'galaxy'), ('galaxy', 'galaxy')]

In [65]:
gg_i=[(i,i) for i in np.arange(zl_bins['n_bins'])]
XI_W_L.stack_indxs[corr_gg]=gg_i
# XI_W_L.stack_indxs

In [66]:
bi=(0,0)

In [67]:
th00,Nb=WT_L.projected_correlation(cl=XI_W_L.SN[corr_ll][:,0,0],l_cl=l0,m1_m2=(2,2))
Nb_b=XI_W_L.binning.bin_1d(xi=Nb,bin_utils=XI_W_L.xi_bin_utils[(2,2)])

AttributeError: 'cov_3X2' object has no attribute 'SN'

In [68]:
plot(thb, Nb_b)
plot(thb, Nb_b*-1)
# plot(th,Nb)
plot(thb, xiWG_L['xi'][corr_ll][(2,2)][bi].compute())
xscale('log')
yscale('log')

NameError: name 'Nb_b' is not defined

In [ ]:
plot(XI_W_L.Win.window_l,XI_W_L.Win.Win['cl'][corr_ll][bi][12]['cl'])
# xscale('log')
yscale('log')

In [76]:
%time XI_W_L.Ang_PS.PS.pk_func(z=[0,1,2,3])

CPU times: user 11.6 s, sys: 1.09 s, total: 12.7 s
Wall time: 9.6 s


(array([[1.31144486e+03, 1.31434952e+03, 1.31726055e+03, ...,
         9.05451097e-01, 9.00827843e-01, 8.96227943e-01],
        [4.85733342e+02, 4.86809674e+02, 4.87888371e+02, ...,
         2.19967159e-01, 2.18792570e-01, 2.17624111e-01],
        [2.29761547e+02, 2.30270840e+02, 2.30781252e+02, ...,
         9.40322823e-02, 9.35373739e-02, 9.30449426e-02],
        [1.31303102e+02, 1.31594219e+02, 1.31885976e+02, ...,
         4.82392423e-02, 4.79979268e-02, 4.77577055e-02]]),
 array([3.00000000e-04, 3.00691710e-04, 3.01385015e-04, ...,
        2.98621350e+01, 2.99309881e+01, 3.00000000e+01]))

In [69]:
# %time xiHG=XI_H.xi_tomo()
# %time xiHG2=XI_H2.xi_tomo()
%time xiWG_L=XI_W_L.xi_tomo()

cl dict done
CPU times: user 14.6 s, sys: 979 ms, total: 15.6 s
Wall time: 10.8 s


In [70]:
%time clG=XI_W_L.cl_tomo()
# %time clH=XI_H.cl_tomo()

cl dict done
CPU times: user 12.8 s, sys: 2.07 s, total: 14.9 s
Wall time: 10.7 s


In [71]:
%time cl=clG['stack'].compute()

CPU times: user 1.37 s, sys: 86.8 ms, total: 1.45 s
Wall time: 105 ms


In [72]:
%time xiW_L=xiWG_L['stack'].compute()

CPU times: user 6.76 s, sys: 207 ms, total: 6.97 s
Wall time: 833 ms


In [73]:
xiWG_L['xi'].keys()

dict_keys([('shear', 'shear'), ('shear', 'galaxy'), ('galaxy', 'galaxy')])

In [75]:
xiW_L['xi'].shape

(900,)

In [42]:
# %time xiH=xiHG['stack'].compute()
#%time xiH2=xiHG2['stack'].compute()
# %time xiH2=xiHG2['stack'].compute()

NameError: name 'xiHG' is not defined

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=0
i=ii*20
j=i+20
ii=0
sn=np.diag(cov_shear)[0]/thb**2*thb[0]**2
cc=xiWG_L['cov'][('shear','shear','shear','shear')][(2,2,2,2)][(ii,ii,ii,ii)].compute()
sn2=np.diag(cc['final'])[0]/thb**2*thb[0]**2
sn2*=1.#4.e5
plot(thb,sn,'r-')
plot(thb,sn2,'ro')
plot(thb,np.diag(cc['final']),'bo')
# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cov_shear)[i:j],label='Cosmolike')
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')
xlabel(r'$\theta$')
ylabel(r'$\delta \xi_+$')
legend()

In [ ]:
XI_W_L.Win.Win['cl'].keys()

In [ ]:
sn2

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=19
i=ii*20
j=i+20
ii=3
sn=0#np.diag(cov_shear)[0]/thb**2*thb[0]**2
cc=xiWG_L['cov'][('shear','shear','shear','shear')][(2,2,2,2)][(ii,ii,ii,ii)].compute()

plot(thb,np.diag(cc['final']),'bo')
# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cov_shear)[i:j],label='Cosmolike')
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')
xlabel(r'$\theta$')
ylabel(r'$\delta \xi_+$')
legend()

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=0
i=ii*20
ii2=1
j=ii2*20

cc2=cov_ggl[i:i+20,j:j+20]
sn=np.diag(cc2)[0]/thb**2*thb[0]**2

cc=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,1,0)].compute()
cc3=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,0,0)].compute()
# cc=xiWG_L['cov'][('galaxy','shear','galaxy','shear')][(0,2,0,2)][(0,0,0,0)].compute()

# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cc['final']),'bo')
# plot(thb,sn,':')
plot(thb,np.diag(cc2))
# plot(thb,np.diag(cc3['final']),'ro')
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=0
i=ii*20
ii2=0
j=ii2*20

cc2=cov_ggl[i:i+20,j:j+20]
sn=np.diag(cc2)[0]/thb**2*thb[0]**2

cc=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,0,0)].compute()


plot(thb,np.diag(cc['final']),'bo')
# plot(thb,sn,':')
plot(thb,np.diag(cov_ggl)[i:i+20])
# plot(thb,np.diag(cc3['final']),'ro')
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
plot(np.tile(thb,20),np.diag(cov_ggl),'bo')
xscale('log')
yscale('log')

In [ ]:
%time xiW_L=xiWG_L['stack'].compute()

In [ ]:
xiW_L['xi'].shape

In [ ]:
f=1
# cov_xiH_inv=np.linalg.inv(xiH['cov']*f)
# print (np.sqrt(np.dot(xiH['xi'],np.dot(cov_xiH_inv,xiH['xi']))))

cov_xiWL_inv=np.linalg.inv(xiW_L['cov']*f)
print (np.sqrt(np.dot(xiW_L['xi'],np.dot(cov_xiWL_inv,xiW_L['xi']))))

In [ ]:
cc=XI_W_L.cov_utils
corrW=cc.corr_matrix(xiW_L['cov'])
pcolor(corrW,vmin=-1,vmax=1,cmap='seismic')
# xlabel(r'$\theta$ (degrees)')
# ylabel(r'$\theta$ (degrees)')
title('Wigner')
# hlines(60,0,120)
# vlines(60,0,120)
xticks([])
yticks([])


colorbar()
show()
# corrL=cc.corr_matrix(clL['cov'])
# pcolor(corrL,vmin=-1,vmax=1,cmap='seismic')

In [ ]:
cc=XI_W_L.cov_utils
corr_shear2=cc.corr_matrix(covG_ggl)
pcolor(corr_shear2,vmin=-1,vmax=1,cmap='seismic')
# xlabel(r'$\theta$ (degrees)')
# ylabel(r'$\theta$ (degrees)')
title('Cosmolike')
# hlines(60,0,120)
# vlines(60,0,120)
xticks([])
yticks([])


colorbar()
show()
# corrL=cc.corr_matrix(clL['cov'])
# pcolor(corrL,vmin=-1,vmax=1,cmap='seismic')

In [ ]:
cc=XI_W_L.cov_utils
corr_shear=cc.corr_matrix(covG_des)
pcolor(corr_shear,vmin=-1,vmax=1,cmap='seismic')
# xlabel(r'$\theta$ (degrees)')
# ylabel(r'$\theta$ (degrees)')
title('Cosmolike')
# hlines(60,0,120)
# vlines(60,0,120)
xticks([])
yticks([])


colorbar()
show()
# corrL=cc.corr_matrix(clL['cov'])
# pcolor(corrL,vmin=-1,vmax=1,cmap='seismic')

In [ ]:
pcolor(corr_shear-corrW,vmin=-1,vmax=1,cmap='seismic')
colorbar()
xticks([])
yticks([])

In [ ]:
pcolor(cov_des/xiW_L['cov']-1,cmap='seismic',vmin=-1,vmax=1,)
colorbar()

In [ ]:
plot(thb,XI_W_L.xi_bin_utils[(2,2)]['norm']/thb**2)
xscale('log')
# yscale('log')

In [ ]:
cov_test=np.genfromtxt('shape_noise_cov.dat')
cov_test2=np.genfromtxt('shape_noise_diag.dat')

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=0
i=ii*20
j=i+20
sn=np.diag(cov_shear)[0]/thb**2*thb[0]**2
# NN=np.diag(xiH['cov'])[i:j]-sn/2.
# plot(thb,np.diag(xiH['cov'])[i:j],'--')
# plot(thb,np.diag(xiH2['cov'])[i:j],'--')
plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
# plot(thb,np.diag(cov_test),':')
# plot(thb,cov_test2,':')
# plot(thb,sn,':')
plot(thb,np.diag(cov_shear)[i:j])
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=1
i=ii*20
j=i+20

cc=xiWG_L['cov'][('galaxy','galaxy','galaxy','galaxy')][(0,0,0,0)][(ii,ii,ii,ii)].compute()

sn=np.diag(cov_gg)[0]/thb**2*thb[0]**2
# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cc['final']),'bo')
# plot(thb,sn,':')
plot(thb,np.diag(cov_gg)[i:j])
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=4
i=ii*20
ii2=0
j=ii2*20

cc2=cov_ggl[i:i+20,j:j+20]
sn=np.diag(cc2)[0]/thb**2*thb[0]**2

cc=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,0,1)].compute()
# cc=xiWG_L['cov'][('galaxy','shear','galaxy','shear')][(0,2,0,2)][(0,0,0,0)].compute()

# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cc['final']),'bo')
# plot(thb,sn,':')
plot(thb,np.diag(cc2))
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
XI_W_L.galaxy_utils.nz_F

In [ ]:
corrs_t=('shear','galaxy','shear','galaxy')
indxs_t=(0,0,0,0)
XI_W_L.cov_utils.get_SN(XI_W_L.SN,corrs_t,indxs_t)

In [ ]:
# plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
# plot(thb,xiH['xi'][:20])
ii=0
i=ii*20
ii2=1
j=ii2*20

cc2=cov_ggl[i:i+20,j:j+20]
sn=np.diag(cc2)[0]/thb**2*thb[0]**2

cc=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,1,0)].compute()
cc3=xiWG_L['cov'][('shear','galaxy','shear','galaxy')][(2,0,2,0)][(0,0,0,0)].compute()
# cc=xiWG_L['cov'][('galaxy','shear','galaxy','shear')][(0,2,0,2)][(0,0,0,0)].compute()

# plot(thb,np.diag(xiW_L['cov'])[i:j],'bo')
plot(thb,np.diag(cc['final']),'bo')
# plot(thb,sn,':')
plot(thb,np.diag(cc2))
# plot(thb,np.diag(cc3['final']),'ro')
# plot(thb,np.diag(cov_shear)[0]/thb**2*thb[0]**2)
xscale('log')
yscale('log')

In [ ]:
plot(thb,np.diag(xiW_L['cov'])[i:j]-cov_test2,'--')
plot(thb,np.diag(xiH['cov'])[i:j]-cov_test2,'--')
plot(thb,(np.diag(cov_shear)[i:j]-cov_test2)*1.2)

xscale('log')
yscale('log')

In [ ]:
plot(thb,np.diag(xiW_L['cov'])[i:j],':')
plot(thb,np.diag(cov_test),':')
plot(thb,cov_test2,':')

xscale('log')
yscale('log')

In [ ]:
plot(thb,np.diag(cov_shear/xiH['cov'])[:20])
plot(thb,np.diag(cov_shear/xiW_L['cov'])[:20])
xscale('log')
# yscale('log')

In [ ]:
d2r/60

In [ ]:
XI_H.z_bins['shear'][0]['nz']

In [ ]:
XI_H.SN

In [ ]:
ns=XI_H.z_bins['shear'][0]['nz']*3600/d2r**2
sigma_gamma**2/ns

In [ ]:
XI_H.z_bins['shear'][0]['SN']

In [ ]:
plot(np.diag(xiW_L['cov']/xiH['cov'])[:20])
ylim(0.9,1.1)

In [ ]:
crash

In [ ]:
for i in np.arange(nbins):
    for j in np.arange(i,nbins):
        lens1 = ccl.ClTracerLensing(ccl_cosmo, False, z=zs_bins[i]['z'], n=zs_bins[i]['pz'])
        lens2 = ccl.ClTracerLensing(ccl_cosmo, False, z=zs_bins[j]['z'], n=zs_bins[j]['pz'])
        cls = ccl.angular_cl(ccl_cosmo, lens1, lens2, CL.l)
        plot(CL.l,clG['cl']['cl'][corr][(i,j)].compute()/cls,label=(i,j))
        xscale('log')
#         yscale('log')
        xlabel(r'$\ell$')
        ylabel(r'$C_\ell$')
        hlines(1,2,1000)
        legend()
        ylim(0.9,1.1)
    show()

In [ ]:
m1m2=(2,2)
cp=['r','b','g','y']#sns.color_pallete()
for i in np.arange(nbins):
    for j in np.arange(i,nbins):
        lens1 = ccl.ClTracerLensing(ccl_cosmo, False, z=zs_bins[i]['z'], n=zs_bins[i]['pz'])
        lens2 = ccl.ClTracerLensing(ccl_cosmo, False, z=zs_bins[j]['z'], n=zs_bins[j]['pz'])
        cls = ccl.angular_cl(ccl_cosmo, lens1, lens2, CL.l)
        cls_b=CL.binning.bin_1d(xi=cls,bin_utils=CL.cl_bin_utils)
        plot(lb,cls_b,color=cp[j],ls=':')
        
    xscale('log')
    yscale('log')
    xlabel(r'$\ell$')
    ylabel(r'$C_\ell$')
    legend(ncol=3)
    show()

In [ ]:
def func(x):
    return x+1

In [ ]:
y=dask.delayed(func)(10)

In [ ]:
y.compute()

In [ ]:
y